Code to make a script to download the spectra from SDSS:

In [1]:
import os
import numpy as np
from astropy.io import fits
from scipy import sparse

In [2]:
tab = fits.getdata("virgo_william.fit")

In [5]:
wget_list = ["wget -c https://dr12.sdss.org/sas/dr12/sdss/spectro/redux/26/spectra/%04d/spec-%04d-%d-%04d.fits\n" % (v["plate"], v["plate"], v["mjd"], v["fiberID"]) for v in tab]

In [6]:
with open("wget_spec.sh", "w") as fp:
    fp.writelines(wget_list)

After spectra is donwnloaded, calculate magnitudes

In [17]:
from syntphot import photoconv
from readfilterset import FilterSet

In [18]:
f = FilterSet("splus_herpich.hdf5")
f.load("splus", 1)

In [29]:
magnitudes = np.zeros((len(tab), len(f.filter_wls)))
specobjids = np.zeros(len(tab), dtype=np.int)

In [56]:
for i_gal, gal in enumerate(tab[:2]):
    v = tab[i_gal]
    specobjids[i_gal] = tab[i_gal]['specObjID']
    magnitudes[i_gal] = photoconv().fromSDSSfits(f.filterset, "spec/spec-%04d-%d-%04d.fits" % (v["plate"], v["mjd"], v["fiberID"]))['m_ab']

Auto-detected table type: fits
N_points: 211, N_bad: 17
Magnitude to filter F0378: 20.60, error: 0.00
N_points: 475, N_bad: 23
Magnitude to filter F0395: 20.69, error: 0.01
N_points: 626, N_bad: 23
Magnitude to filter F0410: 20.56, error: 0.01
N_points: 443, N_bad: 2
Magnitude to filter F0430: 20.27, error: 0.01
N_points: 329, N_bad: 0
No bad pixel! =)
Magnitude to filter F0515: 19.08, error: 0.01
N_points: 257, N_bad: 0
No bad pixel! =)
Magnitude to filter F0660: 18.04, error: 0.01
N_points: 603, N_bad: 32
Magnitude to filter F0861: 17.46, error: 0.00
N_points: 1903, N_bad: 29
Magnitude to filter gSDSS: 19.45, error: 0.00
N_points: 1544, N_bad: 42
Magnitude to filter iSDSS: 17.70, error: 0.00
N_points: 1755, N_bad: 7
Magnitude to filter rSDSS: 18.21, error: 0.00
N_points: 727, N_bad: 23
Magnitude to filter uJAVA: 20.62, error: 0.00
N_points: 952, N_bad: 40
Magnitude to filter zSDSS: 17.43, error: 0.00
Auto-detected table type: fits
N_points: 211, N_bad: 17
Magnitude to filter F0378: 2

In [40]:
specobjids[:2], magnitudes[:2]

(array([347003650687631360, 346722150712868864]),
 array([[ 20.60178947,  20.68842888,  20.56106567,  20.26911545,
          19.08202744,  18.03684807,  17.46020508,  19.45278168,
          17.70477104,  18.20992279,  20.61692238,  17.42697144],
        [ 20.60178947,  20.68842888,  20.56106567,  20.26911545,
          19.08202744,  18.03684807,  17.46020508,  19.45278168,
          17.70477104,  18.20992279,  20.61692238,  17.42697144]]))

In [55]:
try:
    os.unlink("jplus_mags.csv")
except OSError:
    pass
np.savetxt("jplus_mags.csv", np.column_stack((specobjids[:,np.newaxis],magnitudes)), delimiter=',', fmt='%i ' + len(f.filter_wls) * "%6.3f ", header="specobjid " + " ".join(f.filter_wls["ID_filter"]) * 2 )

(numpy.record, [('specObjID', '>i8'), ('mjd', '>i4'), ('plate', '>i2'), ('tile', '>i2'), ('xFocal', '>f8'), ('yFocal', '>f8'), ('fiberID', '>i2'), ('z', '>f4'), ('zErr', '>f4'), ('zConf', '>f4'), ('zStatus', '>i2'), ('specClass', '>i2'), ('zWarning', '>i4'), ('velDisp', '>f4'), ('velDispErr', '>f4'), ('eCoeff_0', '>f4'), ('eCoeff_1', '>f4'), ('eCoeff_2', '>f4'), ('eCoeff_3', '>f4'), ('eCoeff_4', '>f4'), ('eClass', '>f4'), ('nGood', '>i4'), ('sn_0', '>f4'), ('sn_1', '>f4'), ('sn_2', '>f4'), ('mag_0', '>f4'), ('mag_1', '>f4'), ('mag_2', '>f4'), ('ra', '>f8'), ('dec', '>f8'), ('cx', '>f8'), ('cy', '>f8'), ('cz', '>f8'), ('htmID', '>i8'), ('fiberMag_u', '>f4'), ('fiberMag_g', '>f4'), ('fiberMag_r', '>f4'), ('fiberMag_i', '>f4'), ('fiberMag_z', '>f4'), ('primTarget', '>i4'), ('secTarget', '>i4'), ('seguePrimTarget', '>i4'), ('segueSecTarget', '>i4'), ('otherPrimTarget', '>i4'), ('otherSecTarget', '>i4'), ('pixMin', '>i2'), ('pixMax', '>i2'), ('sci_sn', '>f4'), ('smear_sn', '>f4'), ('sm_leg_

In [27]:
print magnitudes[i_gal]

(347003650687631360,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.)


Create a table of starlight_id and specobjid to match using grep

In [3]:
join_tsv = ["%04d.%d.%03d\t %i\n" % (v["plate"], v["mjd"], v["fiberID"], v["specObjID"]) for v in tab]
with open("join_starlight_specobjid.txt", "w") as fp:
    fp.writelines(join_tsv)